# Generate Figure 3 for Kaiser et al. 2020 Science paper

This is the same message at the beginning of all jupyter notebooks in this directory. 

If you don't have the below packages, you obviously need to install them for this to work. If it doesn't work still it's extremely likely you have an outdated version of one of the packages. Alternatively, some of the histogram functions actually rely on not being the most recent version because they changed from "normed" to something else from my recollection. Or perhaps it was the other way. I am aware this was poor decision-making, but it works (if you use the right version). ¯\\_(ツ)_/¯

Also pretty much all of these commands are copied and pasted from another Jupyter notebook I made but contained tons of tries at doing this stuff (and unrelated efforts) so that's why a lot of the variables seem unnecessary to use.

In [1]:


from __future__ import print_function


import matplotlib
matplotlib.use('pdf')


import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from astropy.io import fits
from glob import glob
from astropy.time import Time
from astropy import coordinates as coords
from astropy import units as u
from astropy import constants as const
from astropy import convolution as conv
from astropy.table import Table, Column
import scipy.interpolate as scinterp
import time
start = time.time()

import spec_plot_tools as spt
import cal_params as cp
import plot_spec as ps
import bensby_plotting as bp
import abundance_corrections as acorr
import interp_tau as itau



print(os.getcwd())

No handles with labels found to put in legend.




no file_setting specificied


(116, 4, 27)
(4, 27, 116)
(116,)
(116,)
(27, 116)
(116,)
(116,)
(27, 116)
(116,)
(116,)
(27, 116)
(116,)
(116,)
(27, 116)
/Users/BenKaiser/Desktop/abd1714_Kaiser_et_al_2020_Science


In [2]:
figure_output_dir='/Users/BenKaiser/Desktop/GaiaJ1644m0449_paper/Science_versions/Third_Revision/figures'

In [3]:
target_dir= '/Users/BenKaiser/Desktop/GaiaJ1644m0449_paper/'
os.chdir(target_dir)

In [4]:
wd_abund_file='temp_wd_abundances.csv'
lodders_abund_file='Lodders2009_solarsystem_abundances.csv'

In [5]:
wd_abund_table=Table.read(wd_abund_file)
wd_abund_table.add_index('name')

In [6]:
lodders_table=Table.read(lodders_abund_file)
lodders_table.add_index('element')
solar_lica=lodders_table.loc['Li']['A_el']-lodders_table.loc['Ca']['A_el']

In [7]:
color_dict={
    'GaiaJ1644-0449':'r',
    'SDSSJ1330+6435':'darkviolet',
    'WDJ2356-209':'#44b839'
}
step_dict={
    'GaiaJ1644-0449':10,
    'SDSSJ1330+6435':0.1,
    'WDJ2356-209':10
}

wd_marker='*'
met_marker='D'
met_color='#1ca1f2'
ci_size=10
ci_leg_size=6
wd_size=12
starsize=4
dp_alpha=0.5
arr_naca=[-0.4,-0.1]
alpha_range=[0.5,0.2]
arrow_segs=100
arrow_width=0.07
arrow_line=4
legend_font=7
star_marker='o'
pop_colors=['darkorange','brown','navy','grey'] #thin disk, thick disk, halo, in-between for Bensby plots




In [8]:

def plot_lica_age(row,sol_norm=True,lica_string='li/ca', color=None, use_central_vals=True,SSP=True,t_step_units='Ca',add_arrow=True,t_step=5,t_max=5):
    print('\n\n',row['name'],'\n\n')
    age_errs= [[row['age_minus']],[row['age_plus']]]
    #age_errs= row['age_plus'],row['age_minus']
    #age_errs=[row['age_minus'],row['age_plus']]
    lica_err= row[lica_string+'_err']
    target_row=row
    target_lica=row[lica_string]
    string1= lica_string
    string2="na/ca" #we need a placeholder for the declining phase function for a second abundance ratio even though we won't use it
    #print('age_errs',age_errs.shape)
    #plt.errorbar(row['age'],row[lica_string],xerr=row['age_minus'],label=row['name'],marker='o')
    if SSP:
        target_lica, target_kca, lica_err, kca_err=acorr.easy_dist_ssp(target_row,['Li','Ca','Na'], plot_all=False,tau_rand=True)
        if row['name']=='WDJ2356-209':
             #we're no longer plotting this one since we don't think it's a CI chondrite
            if sol_norm:
                pass
                #plt.errorbar(float(row['age']),target_lica-solar_lica,marker=wd_marker, label=row['name'],color=color_dict[row['name']],markersize=wd_size, linestyle='None')
                #plt.errorbar(float(row['age']),target_lica-solar_lica,xerr=age_errs,yerr=0.3,marker=wd_marker, uplims=True,color=color_dict[row['name']],markersize=wd_size, linestyle='None')
                #plt.ylabel('[Li/Ca]')
            else:
                pass
                #plt.errorbar(float(row['age']),target_lica,marker=wd_marker, label=row['name'],color=color_dict[row['name']],markersize=wd_size, linestyle='None')
                #plt.errorbar(float(row['age']),target_lica,xerr=age_errs,yerr=0.3,marker=wd_marker, uplims=True,color=color_dict[row['name']],markersize=wd_size, linestyle='None')
        else:
            if sol_norm:
                plt.errorbar(float(row['age']),target_lica-solar_lica,xerr=age_errs,label=spt.fix_display_string(row['name']),yerr=lica_err,marker=met_marker, color=color_dict[row['name']],markersize=ci_size, linestyle='None')
                plt.ylabel('[Li/Ca]')
            else:
                plt.errorbar(float(row['age']),target_lica,xerr=age_errs,yerr=lica_err,marker=met_marker,color=color_dict[row['name']],markersize=ci_size, linestyle='None')
                plt.errorbar(float(row['age']),target_lica,xerr=age_errs,label=spt.fix_display_string(row['name'])+' SSP',yerr=lica_err,marker=met_marker,color=color_dict[row['name']],markersize=ci_leg_size, linestyle='None')
                if add_arrow:
                    tau_time= 10.**itau.extrapolate_tau_x_logg(row['teff'], row['logg'], t_step_units)
                    tau_time=tau_time*1e-6 #converted to Myr
                    t_step=t_step*tau_time
                    t_max=t_max*tau_time
                    times=t_step
                    arrow_endy,throwaway=acorr.el1el2_DP_el3el2_ftimes(row['teff'], row[string1], row[string2],times, 'Li', 'Ca', 'Na', logg=row['logg'])
                    arrow_endx=float(row['age'])
                    #print('arrow_endy',arrow_endy,'arrow_endx',arrow_endx)
                    #plt.plot(arrow_endx,arrow_endy,marker='o')
                    ypoints=np.linspace(target_lica,arrow_endy,arrow_segs)
                    xpoints=np.linspace(float(row['age']),arrow_endx,arrow_segs)
                    #dx=arrow_endx-target_el3el2
                    #dy=arrow_endy-target_el1el2
                    dx=arrow_endx-xpoints[-2]
                    dy=arrow_endy-ypoints[-2]
                    def get_segs(points):
                        return np.vstack([points,np.roll(points,1)]).T[1:]
                    x_segs=get_segs(xpoints)
                    y_segs=get_segs(ypoints)
                    #print('x_segs',x_segs)
                    #color_array=np.empty_like(ypoints,dtype=str)
                    #color_array[:]=color_dict[name]
                    alpha_vals=np.linspace(alpha_range[0],alpha_range[1],arrow_segs)
                    #print('alpha_vals',alpha_vals)
                    #for x,y,alpha in zip(x_segs, y_segs,alpha_vals):
                        #plt.plot(x,y,color=color_dict[name],alpha=alpha,linewidth=arrow_line)
                        #plt.plot(x,y,color=color_dict[row['name']],alpha=alpha,linewidth=arrow_line)
                        #plt.plot(x,y,color=color_dict[row['name']],alpha=alpha,linewidth=arrow_line)
                    #plt.arrow(xpoints[-2],ypoints[-2],dx,dy,color=color_dict[row['name']],width=arrow_width, alpha=alpha_range[1],length_includes_head=True )
                    plt.arrow(float(row['age']),target_lica,arrow_endx-float(row['age']),arrow_endy-target_lica,color=color_dict[row['name']],width=arrow_width,alpha=alpha_range[0],length_includes_head=True)
                else:
                    pass
                plt.ylabel(r'log(Li/Ca)')
    elif use_central_vals:
        if row['name']=='WDJ2356-209':
            if sol_norm:
                pass
                #plt.errorbar(float(row['age']),row[lica_string]-solar_lica,marker=wd_marker, label=row['name'],color=color_dict[row['name']],markersize=wd_size, linestyle='None')
                #plt.errorbar(float(row['age']),row[lica_string]-solar_lica,xerr=age_errs,yerr=0.3,marker=wd_marker, uplims=True,color=color_dict[row['name']],markersize=wd_size, linestyle='None')
                #plt.ylabel('[Li/Ca]')
            else:
                pass
                #plt.errorbar(float(row['age']),row[lica_string],marker=wd_marker, label=row['name'],color=color_dict[row['name']],markersize=wd_size, linestyle='None')
                #plt.errorbar(float(row['age']),row[lica_string],xerr=age_errs,yerr=0.3,marker=wd_marker, uplims=True,color=color_dict[row['name']],markersize=wd_size, linestyle='None')
        else:
            if sol_norm:
                plt.errorbar(float(row['age']),row[lica_string]-solar_lica,xerr=age_errs,label=spt.fix_display_string(row['name']),yerr=lica_err,marker=wd_marker, color=color_dict[row['name']],markersize=wd_size, linestyle='None')
                plt.ylabel('[Li/Ca]')
            else:
                plt.errorbar(float(row['age']),row[lica_string],xerr=age_errs,label=spt.fix_display_string(row['name']),yerr=lica_err,marker=wd_marker,color=color_dict[row['name']],markersize=wd_size, linestyle='None')
                plt.ylabel(r'log(Li/Ca)')
    else:
        #doing upper limits only because we don't know the phase.
        print("doing upper limits exclusively... shouldn't have happened")
        lica_val=row[lica_string]+3*lica_err #3-sigma upper side of the distribution
        plt.errorbar(float(row['age']),lica_val,marker=wd_marker, label=row['name'],color=color_dict[row['name']],markersize=wd_size, linestyle='None')
        plt.errorbar(float(row['age']),lica_val,xerr=age_errs,yerr=0.3,marker=wd_marker,uplims=True,color=color_dict[row['name']],markersize=wd_size, linestyle='None')
        plt.ylabel(r'$\log$(Li/Ca)')
    return

In [9]:
#plt.figure(figsize=(15,10))
spt.initiate_science_plot()
plt.figure(figsize=(4.75,4.75),constrained_layout=True)



count=0
bp.plot_lica_age_pop(colors=pop_colors, rep_errors=True, sol_norm=False,markersize=starsize)
plt.errorbar(4.57, 1.10-6.33, yerr=np.sqrt(0.1**2+0.07**2),marker=star_marker, color=met_color, label="Sun",linestyle='None',markersize=starsize )
plt.errorbar(4.57,lodders_table.loc['Li']['A_el']-lodders_table.loc['Ca']['A_el'],yerr=np.sqrt(lodders_table.loc['Li']['A_el_err']**2+lodders_table.loc['Ca']['A_el_err']**2),label="CI Chondrites",marker=met_marker, color=met_color, linestyle='None', markersize=ci_leg_size)
plt.errorbar(4.57,lodders_table.loc['Li']['A_el']-lodders_table.loc['Ca']['A_el'],yerr=np.sqrt(lodders_table.loc['Li']['A_el_err']**2+lodders_table.loc['Ca']['A_el_err']**2),marker=met_marker,markersize=ci_size, color=met_color)
#plt.xlim(14,0)

for row in wd_abund_table:
    if count > 2:
        break
    #plot_lica_age(row,sol_norm=False,lica_string='li/ca', use_central_vals=False)
    plot_lica_age(row,sol_norm=False,lica_string='li/ca', use_central_vals=True,SSP=False)
    plot_lica_age(row,sol_norm=False,lica_string='li/ca', use_central_vals=True,SSP=True)
    #if row['name']=='WDJ2356-209':
        
    count+=1

plt.xlim(15,0)


plt.axvline(x=13.8, linestyle=':', color='k')
plt.legend(fontsize=legend_font)
#plt.grid(True)
plt.xlabel('Age (Gyr)')



print(os.getcwd())
os.chdir(figure_output_dir)
print(os.getcwd())
start = time.time()
print(start)
time_string=str(start).split('.')[0]
plt.savefig('figure3_'+time_string+'.pdf')#plt.grid(True)

plt.show()

       B_ALi        
--------------------
0.030000000000000027
 0.10000000000000009
 0.06000000000000005
 0.06000000000000005
 0.04999999999999982
 0.06000000000000005
 0.04999999999999982
                  --
0.039999999999999813
0.050000000000000266
                 ...
 0.06000000000000005
0.040000000000000036
                  --
0.040000000000000036
 0.06000000000000005
                  --
0.040000000000000036
 0.08999999999999986
                  --
                  --
                0.06
Length = 714 rows
<class 'astropy.table.column.MaskedColumn'>
star_age.shape (714,)
in_range (array([  5,  11,  13,  14,  34,  36,  39,  47,  64,  74,  83, 109, 112,
       134, 137, 151, 158, 163, 185, 186, 245, 273, 274, 277, 283, 295,
       318, 331, 335, 340, 368, 420, 438, 442, 469, 472, 478, 488, 495,
       496, 520, 590, 593, 596, 600, 606, 613, 621, 629, 636, 638, 641,
       643, 649, 664, 687, 689]),)
(714,)
max_index 368
max_lica -5.75
max age 4.0
med_age_error [1.9 1. ]
med_lic